In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gc
import json
import os
import shutil

import data_formatters.base
import libs.utils as utils
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
# Layer definitions.
concat = tf.keras.backend.concatenate
stack = tf.keras.backend.stack
K = tf.keras.backend
Add = tf.keras.layers.Add
LayerNorm = tf.keras.layers.LayerNormalization
Dense = tf.keras.layers.Dense
Multiply = tf.keras.layers.Multiply
Dropout = tf.keras.layers.Dropout # Inputs elements are randomly set to zero (and the other elements are rescaled)
Activation = tf.keras.layers.Activation
Lambda = tf.keras.layers.Lambda

# Default input types.
InputTypes = data_formatters.base.InputTypes

In [3]:
import abc
import enum
class DataTypes(enum.IntEnum):
  """Defines numerical types of each column."""
  REAL_VALUED = 0
  CATEGORICAL = 1
  DATE = 2

class InputTypes(enum.IntEnum):
  """Defines input types of each column."""
  TARGET = 0
  OBSERVED_INPUT = 1
  KNOWN_INPUT = 2
  STATIC_INPUT = 3
  ID = 4  # Single column used as an entity identifier
  TIME = 5 

In [4]:
import pickle 
with open('params_update.pkl', 'rb') as f:
    params = pickle.load(f)

In [5]:
params

{'dropout_rate': 0.1,
 'hidden_layer_size': 5,
 'learning_rate': 0.001,
 'max_gradient_norm': 100.0,
 'minibatch_size': 128,
 'model_folder': '0615_result',
 'num_heads': 4,
 'stack_size': 1,
 'total_time_steps': 120,
 'num_encoder_steps': 90,
 'num_epochs': 1,
 'early_stopping_patience': 5,
 'multiprocessing_workers': 5,
 'column_definition': [('traj_id',
   <DataTypes.REAL_VALUED: 0>,
   <InputTypes.ID: 4>),
  ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>),
  ('log_sales', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>),
  ('transactions', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>),
  ('oil', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>),
  ('day_of_month', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
  ('month', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
  ('open', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
  ('item_nbr', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>),
  ('store_nbr', <Data

In [6]:
# Data parameters
time_steps = int(params['total_time_steps'])
input_size = int(params['input_size'])
output_size = int(params['output_size'])
category_counts = json.loads(str(params['category_counts'])) 
n_multiprocessing_workers = int(params['multiprocessing_workers'])

In [7]:
# Relevant indices for TFT
_input_obs_loc = json.loads(str(params['input_obs_loc']))
_static_input_loc = json.loads(str(params['static_input_loc']))
_known_regular_input_idx = json.loads(
        str(params['known_regular_inputs']))
_known_categorical_input_idx = json.loads(
        str(params['known_categorical_inputs']))
    # json.loads: parse a valid JSON string and convert it into a Python Dictionary
column_definition = params['column_definition']

In [8]:
# Network params
quantiles = [0.1, 0.5, 0.9]
use_cudnn = False  # Whether to use GPU optimised LSTM
hidden_layer_size = int(params['hidden_layer_size'])
dropout_rate = float(params['dropout_rate'])
max_gradient_norm = float(params['max_gradient_norm'])
learning_rate = float(params['learning_rate'])
minibatch_size = int(params['minibatch_size'])
num_epochs = int(params['num_epochs'])
early_stopping_patience = int(params['early_stopping_patience'])

num_encoder_steps = int(params['num_encoder_steps'])
num_stacks = int(params['stack_size'])
num_heads = int(params['num_heads'])

In [9]:
# Serialisation options
_temp_folder = os.path.join(params['model_folder'], 'tmp')
print('Resetting temp folder...')
utils.create_folder_if_not_exist(_temp_folder)
shutil.rmtree(_temp_folder)
os.makedirs(_temp_folder)

# Extra components to store Tensorflow nodes for attention computations
_input_placeholder = None
_attention_components = None
_prediction_parts = None

#print('*** {} params ***'.format())
for k in params:
    print('# {} = {}'.format(k, params[k]))

Resetting temp folder...
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = 0615_result
# num_heads = 4
# stack_size = 1
# total_time_steps = 120
# num_encoder_steps = 90
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('traj_id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_sales', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('transactions', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('oil', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('day_of_month', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('month', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('open', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('item_nbr', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>), ('store_nbr', <DataTypes.CATEGORI

### Embedding

##### define function

In [10]:
# Embedding
num_categorical_variables = len(category_counts) # num_categorical_variables=14
num_regular_variables = input_size - num_categorical_variables # num_regular_variables=6

In [11]:
## def _build_base_graph(self):
# Inputs.
all_inputs = tf.compat.v1.keras.layers.Input(
        shape=(
            time_steps, 
            input_size,
        )) # shape=(None, 120, 20)
regular_inputs, categorical_inputs \
        = all_inputs[:, :, :num_regular_variables],\
          all_inputs[:, :, num_regular_variables:]
# regular_inputs: shape=(None, 120, 6)
# categorical_inputs: shape=(None, 120, 14)

In [12]:
## (1) Embedding for Numerical variables
def convert_real_to_embedding(x):
      """Applies linear transformation for time-varying inputs."""
      return tf.keras.layers.TimeDistributed(
          tf.keras.layers.Dense(hidden_layer_size))(
              x)

In [13]:
## (2) Embedding for categorical variables
embedding_sizes = [
        hidden_layer_size for i, size in enumerate(category_counts)
    ]
embeddings = []
for i in range(num_categorical_variables):
      embedding = tf.keras.Sequential([
          tf.keras.layers.InputLayer([time_steps]),
          tf.keras.layers.Embedding(
              category_counts[i], # input_dim
              embedding_sizes[i], # output_dim , embedding_sizes[i]=hidden_layer_size
              input_length=time_steps,
              dtype=tf.float32)
      ]) 
      embeddings.append(embedding)

tf.keras.layers.InputLayer([time_steps]) defines an input layer for a neural network model. 

The time_steps parameter specifies the shape of the input data, indicating how many time steps the model will process at once, which is especially useful for sequence data like time series.

In [14]:
# categorical_inputs转化为embedded_inputs
### categorical_inputs: shape=(None, 120, 14)
### embedded_inputs: 14个shape=(None, 120, 5)
embedded_inputs = [
        embeddings[i](categorical_inputs[Ellipsis, i]) # categorical_inputs[Ellipsis, i]: shape=(None, 120)
        for i in range(num_categorical_variables)
    ]

In [15]:
embedded_inputs

[<tf.Tensor 'sequential/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_1/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_2/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_3/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_4/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_5/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_6/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_7/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_8/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_9/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_10/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_11/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_12/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_

when a list of keras tensors that each has 3-dim, if we want to concat along last axis:
def keras_concat_last_axis(t1):
    temp_concat_swapaxes=[]
    for i in np.arange(len(t1)):
        temp_concat_swapaxes.append(tf.keras.ops.swapaxes((t1)[i],1,2))
    stack_temp = tf.keras.ops.hstack(temp_concat_swapaxes)
    result = tf.keras.ops.swapaxes(stack_temp,1,2)
    return result

#### Static inputs

In [16]:
# (1) Static inputs: shape=(None, 9, 5)
# static_inputs = [Dense(for each regular_inputs) + embedded_inputs]
if _static_input_loc:
      static_inputs = [tf.keras.layers.Dense(hidden_layer_size)(
          regular_inputs[:, 0, i:i + 1]) for i in range(num_regular_variables) # dim(1)=0
                       if i in _static_input_loc] \
            + [embedded_inputs[i][:, 0, :] # dim(1)=0
             for i in range(num_categorical_variables)
             if i + num_regular_variables in _static_input_loc]
      
      static_inputs = stack(static_inputs, axis=1) # concat base on axis=1

else:
      static_inputs = None

In [17]:
static_inputs

<tf.Tensor 'stack:0' shape=(None, 9, 5) dtype=float32>

In [18]:
for i in range(num_categorical_variables):
    if i + num_regular_variables in _static_input_loc:
        #static_inputs = [embedded_inputs[i][:, 0, :]]
        print([embedded_inputs[i][:, 0, :]])
        #static_inputs


[<tf.Tensor 'strided_slice_25:0' shape=(None, 5) dtype=float32>]
[<tf.Tensor 'strided_slice_26:0' shape=(None, 5) dtype=float32>]
[<tf.Tensor 'strided_slice_27:0' shape=(None, 5) dtype=float32>]
[<tf.Tensor 'strided_slice_28:0' shape=(None, 5) dtype=float32>]
[<tf.Tensor 'strided_slice_29:0' shape=(None, 5) dtype=float32>]
[<tf.Tensor 'strided_slice_30:0' shape=(None, 5) dtype=float32>]
[<tf.Tensor 'strided_slice_31:0' shape=(None, 5) dtype=float32>]
[<tf.Tensor 'strided_slice_32:0' shape=(None, 5) dtype=float32>]
[<tf.Tensor 'strided_slice_33:0' shape=(None, 5) dtype=float32>]


#### Targets

In [19]:
def convert_real_to_embedding(x):
      """Applies linear transformation for time-varying inputs."""
      return tf.keras.layers.TimeDistributed(
          tf.keras.layers.Dense(hidden_layer_size))(
              x)

In [20]:
# (2) Targets: shape=(None, 120, 5, 1)
# obs_inputs = TimeDistributed(Dense())(regular_inputs)  
obs_inputs = tf.keras.backend.stack([
        convert_real_to_embedding(regular_inputs[Ellipsis, i:i + 1]) # for ith regular variables
        for i in _input_obs_loc
    ],axis=-1)
# regular_inputs[Ellipsis, i:i + 1]: (None, 120, 1)

In [21]:
for i in _input_obs_loc:

    print(tf.keras.layers.TimeDistributed(

          tf.keras.layers.Dense(hidden_layer_size))(
            
              regular_inputs[Ellipsis, 0:1]))

Tensor("time_distributed_1/Identity:0", shape=(None, 120, 5), dtype=float32)


#### Observed (a priori unknown) inputs

In [22]:
# (3) Observed (a priori unknown) inputs: shape=(None, 120, 5, 2)
# unknown_inputs = unknown_inputs + wired_embeddings
wired_embeddings = [] # for categorical & not belongs to known & not belongs to _input_obs_loc
for i in range(num_categorical_variables):
      if i not in _known_categorical_input_idx \
        and  i + num_regular_variables  not in _input_obs_loc:
        e = embeddings[i](categorical_inputs[:, :, i])
        wired_embeddings.append(e)

unknown_inputs_temp = [] # for regular_inputs & not belongs to known & not belongs to _input_obs_loc
for i in range(regular_inputs.shape[-1]):
      if i not in _known_regular_input_idx \
          and i not in _input_obs_loc:
        e = convert_real_to_embedding(regular_inputs[Ellipsis, i:i + 1])
        unknown_inputs_temp.append(e)

In [23]:
if unknown_inputs_temp + wired_embeddings:
      unknown_inputs = tf.keras.backend.stack(unknown_inputs_temp + wired_embeddings, axis=-1)
else:
      unknown_inputs = None

In [24]:
unknown_inputs

<tf.Tensor 'stack_2:0' shape=(None, 120, 5, 2) dtype=float32>

In [25]:
unknown_inputs_temp + wired_embeddings

[<tf.Tensor 'time_distributed_2/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'time_distributed_3/Identity:0' shape=(None, 120, 5) dtype=float32>]

#### A priori known inputs

In [26]:
# (4) A priori known inputs
    # known_combined_layer = known_regular_inputs + known_categorical_inputs

known_regular_inputs = [ # for _known_regular & not belongs to _static_input
        convert_real_to_embedding(regular_inputs[Ellipsis, i:i + 1])
        for i in _known_regular_input_idx
        if i not in _static_input_loc
    ] 
known_categorical_inputs = [ # for _known_categorical & & not belongs to _static_input
        embedded_inputs[i]
        for i in _known_categorical_input_idx
        if i + num_regular_variables not in _static_input_loc
    ]

known_combined_layer = tf.keras.backend.stack(known_regular_inputs + known_categorical_inputs, axis=-1)

In [27]:
known_combined_layer

<tf.Tensor 'stack_3:0' shape=(None, 120, 5, 8) dtype=float32>

In [28]:
known_regular_inputs + known_categorical_inputs

[<tf.Tensor 'time_distributed_4/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'time_distributed_5/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'time_distributed_6/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_9/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_10/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_11/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_12/Identity:0' shape=(None, 120, 5) dtype=float32>,
 <tf.Tensor 'sequential_13/Identity:0' shape=(None, 120, 5) dtype=float32>]

### Isolate known and observed historical inputs.

In [29]:
known_combined_layer[:, num_encoder_steps:, :]

<tf.Tensor 'strided_slice_41:0' shape=(None, 30, 5, 8) dtype=float32>

In [30]:
# Isolate known and observed historical inputs.
if unknown_inputs is not None:
      historical_inputs = concat([
          unknown_inputs[:, :num_encoder_steps, :], # shape=(None, 90, 5, 2)
          known_combined_layer[:, :num_encoder_steps, :], # shape=(None, 90, 5, 8)
          obs_inputs[:, :num_encoder_steps, :] # shape=(None, 90, 5, 1)
      ],axis=-1
      )
else:
      historical_inputs = concat([
          known_combined_layer[:, :num_encoder_steps, :],
          obs_inputs[:, :num_encoder_steps, :]
      ],axis=-1
      )
# historical_inputs.get_shape().as_list() = _, time_steps, embedding_dim, num_inputs

# Isolate only known future inputs.
future_inputs = known_combined_layer[:, num_encoder_steps:, :] # shape=(None, 30, 5, 8)

In [31]:
historical_inputs

<tf.Tensor 'concat:0' shape=(None, 90, 5, 11) dtype=float32>

In [32]:
future_inputs

<tf.Tensor 'strided_slice_45:0' shape=(None, 30, 5, 8) dtype=float32>

### Module
#### Basic Module
##### linear_layer

In [33]:
def linear_layer(size,
                 activation=None,
                 use_time_distributed=False,
                 use_bias=True):
  """Returns simple Keras linear layer.

  Args:
    size: Output size
    activation: Activation function to apply if required
    use_time_distributed: Whether to apply layer across time
    use_bias: Whether bias should be included in layer
  """
  linear = tf.keras.layers.Dense(size, activation=activation, use_bias=use_bias)
  if use_time_distributed:
    linear = tf.keras.layers.TimeDistributed(linear)
  return linear

##### add_and_norm

In [34]:
def add_and_norm(x_list):
  """Applies skip connection followed by layer normalisation.

  Args:
    x_list: List of inputs to sum for skip connection

  Returns:
    Tensor output from layer.
  """
  tmp = Add()(x_list)   #tmp = tf.keras.layers.Add(x1,x2)
  tmp = LayerNorm()(tmp)
  return tmp

##### MLP

In [35]:
def apply_mlp(inputs,
              hidden_size,
              output_size,
              output_activation=None,
              hidden_activation='tanh',
              use_time_distributed=False):
  """Applies simple feed-forward network to an input.

  Args:
    inputs: MLP inputs
    hidden_size: Hidden state size
    output_size: Output size of MLP
    output_activation: Activation function to apply on output
    hidden_activation: Activation function to apply on input
    use_time_distributed: Whether to apply across time

  Returns:
    Tensor for MLP outputs.
  """
  if use_time_distributed:
    hidden = tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(hidden_size, activation=hidden_activation))(
            inputs)
    return tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(output_size, activation=output_activation))(
            hidden)
  else:
    hidden = tf.keras.layers.Dense(
        hidden_size, activation=hidden_activation)(
            inputs)
    return tf.keras.layers.Dense(
        output_size, activation=output_activation)(
            hidden)

##### Gate

In [36]:
def apply_gating_layer(x,
                       hidden_layer_size,
                       dropout_rate=None,
                       use_time_distributed=True,
                       activation=None):
  """Applies a Gated Linear Unit (GLU) to an input.

  Args:
    x: Input to gating layer
    hidden_layer_size: Dimension of GLU
    dropout_rate: Dropout rate to apply if any
    use_time_distributed: Whether to apply across time
    activation: Activation function to apply to the linear feature transform if
      necessary

  Returns:
    Tuple of tensors for: (GLU output, gate)
  """
  # First, dropout
  if dropout_rate is not None:
    x = tf.keras.layers.Dropout(dropout_rate)(x)

  if use_time_distributed:
    activation_layer = tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(hidden_layer_size, activation=activation))(
            x)
    gated_layer = tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'))(
            x)
  else:
    activation_layer = tf.keras.layers.Dense(
        hidden_layer_size, activation=activation)(
            x)
    gated_layer = tf.keras.layers.Dense(
        hidden_layer_size, activation='sigmoid')(
            x)

  #return tf.keras.layers.Multiply()([activation_layer, gated_layer]), gated_layer
  return activation_layer, gated_layer

##### GRN

In [37]:
def gated_residual_network(x,   # primary input
                           hidden_layer_size,
                           output_size=None,
                           dropout_rate=None,
                           use_time_distributed=True,
                           additional_context=None,
                           return_gate=False):
  """Applies the gated residual network (GRN) as defined in paper.

  Args:
    x: Network inputs
    hidden_layer_size: Internal state size
    output_size: Size of output layer
    dropout_rate: Dropout rate if dropout is applied
    use_time_distributed: Whether to apply network across time dimension
    additional_context: Additional context vector to use if relevant
    return_gate: Whether to return GLU gate for diagnostic purposes

  Returns:
    Tuple of tensors for: (GRN output, GLU gate)
  """

  # Setup skip connection
  if output_size is None:
    output_size = hidden_layer_size
    skip = x
  else: # skip = TimeDistributed(Dense(output_size))(x)
    skip = linear_layer(
        output_size,
        activation=None,
        use_time_distributed=use_time_distributed
    )(x)    
    # original edition:
    #linear = Dense(output_size)
    #if use_time_distributed:
      #linear = tf.keras.layers.TimeDistributed(linear)
    #skip = linear(x)


  # Apply feedforward network
  #### (1) Dense
  hidden = linear_layer(
      hidden_layer_size,
      activation=None,
      use_time_distributed=use_time_distributed)(
          x)
  if additional_context is not None:
    hidden = hidden + linear_layer(
        hidden_layer_size,
        activation=None,
        use_time_distributed=use_time_distributed,
        use_bias=False)(
            additional_context)
  #### (2) ELU
  hidden = Activation('elu')(hidden)

  #### (3) Dense
  hidden = linear_layer(
      hidden_layer_size,
      activation=None,
      use_time_distributed=use_time_distributed)(
          hidden)
  #### (4) Gate: GLU
  gating_layer, gate = apply_gating_layer(
      hidden,
      output_size,
      dropout_rate=dropout_rate,
      use_time_distributed=use_time_distributed,
      activation=None)
  #### (5) Add & Norm
  if return_gate:
    return add_and_norm([skip, gating_layer]), gate
    #return add_and_norm(skip, hidden), gate
  else:
    return add_and_norm([skip, gating_layer])

  gating_layer, gate = apply_gating_layer(
      hidden,
      output_size,
      dropout_rate=dropout_rate,
      use_time_distributed=use_time_distributed,
      activation=None)

#### Model
##### Variable selection for static inputs

In [90]:
# static_inputs: shape should be (None, 9, 5)
num_static = static_inputs.shape[1] # num_static = 9

# (1) sparse_weights: shape=(None, 9, 1)
flatten = tf.keras.layers.Flatten()(static_inputs) # shape=(None, 45) 9*5=45
mlp_outputs = gated_residual_network(    # shape=(None, 9)
          flatten,
          hidden_layer_size,
          output_size=num_static,
          dropout_rate=dropout_rate,
          use_time_distributed=False,
          additional_context=None)
sparse_weights = tf.keras.layers.Activation('softmax')(mlp_outputs)
sparse_weights = tf.keras.backend.expand_dims(sparse_weights, axis=-1) # tf.keras.backend.expand_dims:在某个位置多加一个维度，数值为1

In [98]:
# static_inputs: shape should be (None, 9, 5)
num_static = static_inputs.shape[1] # num_static = 9

# (1) sparse_weights: shape=(None, 9, 1)
flatten = tf.keras.layers.Flatten()(static_inputs) # shape=(None, 45) 9*5=45
mlp_outputs = gated_residual_network(    # shape=(None, 9)
          flatten,
          hidden_layer_size,
          output_size=num_static,
          dropout_rate=dropout_rate,
          use_time_distributed=False,
          additional_context=None)
sparse_weights = tf.keras.layers.Activation('softmax')(mlp_outputs)
sparse_weights = tf.keras.backend.expand_dims(sparse_weights, axis=-1) # tf.keras.backend.expand_dims:在某个位置多加一个维度，数值为1

# (2) transformed_embedding: shape=(None, 9, 5)
trans_emb_list = []
for i in range(num_static):
        e = gated_residual_network(
            static_inputs[:, i:i + 1, :], # static_inputs=(None, 9, 5)
            hidden_layer_size,
            dropout_rate=dropout_rate,
            use_time_distributed=False)
        trans_emb_list.append(e) # e: shape=(None, 1, 5)
transformed_embedding = concat(trans_emb_list, axis=1)

# combined: shape=(None, 9, 5)
combined = tf.keras.layers.Multiply()(
          [sparse_weights, transformed_embedding])
static_vec = K.sum(combined, axis=1) # K = tf.keras.backend # shape=(None, 5)

In [39]:
static_encoder = static_vec
static_weights = sparse_weights
print(static_encoder)
print(static_weights)

Tensor("Sum:0", shape=(None, 5), dtype=float32)
Tensor("ExpandDims:0", shape=(None, 9, 1), dtype=float32)


##### Static covariates Encoders(4 kinds)

In [40]:
# shape=(None, 5), four identical static variables 
static_context_variable_selection = gated_residual_network(
        static_encoder,
        hidden_layer_size,
        dropout_rate=dropout_rate,
        use_time_distributed=False)
static_context_enrichment = gated_residual_network(
        static_encoder,
        hidden_layer_size,
        dropout_rate=dropout_rate,
        use_time_distributed=False)
static_context_state_h = gated_residual_network(
        static_encoder,
        hidden_layer_size,
        dropout_rate=dropout_rate,
        use_time_distributed=False)
static_context_state_c = gated_residual_network(
        static_encoder,
        hidden_layer_size,
        dropout_rate=dropout_rate,
        use_time_distributed=False)

In [41]:
static_context_state_h

<tf.Tensor 'layer_normalization_12/Identity:0' shape=(None, 5) dtype=float32>

##### Variable selection for other inputs
###### (1) Historical data

In [42]:
# (1) Historical data
historical_inputs

<tf.Tensor 'concat:0' shape=(None, 90, 5, 11) dtype=float32>

In [43]:
_, historical_time_steps, embedding_dim, num_inputs = historical_inputs.get_shape().as_list()
#Wrong edition:
# _, historical_time_steps, embedding_dim, num_inputs = tf.keras.backend.shape(historical_inputs)
flatten = tf.keras.backend.reshape(historical_inputs,
                          [-1, historical_time_steps, embedding_dim * num_inputs]) # shape=(None, 90, 55)
expanded_static_context = tf.keras.backend.expand_dims(
          static_context_variable_selection, axis=1) # shape=(None, 1, 5)

# Variable selection weights
mlp_outputs, static_gate = gated_residual_network(
          flatten,
          hidden_layer_size,
          output_size=num_inputs,
          dropout_rate=dropout_rate,
          use_time_distributed=True,
          additional_context=expanded_static_context, # here
          return_gate=True) # mlp_outputs: shape=(None, 90, 11); static_gate: shape=(None, 90, 11)


sparse_weights = tf.keras.layers.Activation('softmax')(mlp_outputs)
sparse_weights = tf.expand_dims(sparse_weights, axis=2) # shape=(None, 90, 1, 11)

# Non-linear Processing & weight application
trans_emb_list = []
for i in range(num_inputs):
        grn_output = gated_residual_network(
            historical_inputs[Ellipsis, i],
            hidden_layer_size,
            dropout_rate=dropout_rate,
            use_time_distributed=True)
        trans_emb_list.append(grn_output) # 11个shape=(None, 90, 5)
transformed_embedding = tf.keras.backend.stack(trans_emb_list, axis=-1) # shape=(None, 90, 5, 11)

combined = tf.keras.layers.Multiply()(
          [sparse_weights, transformed_embedding]) # shape=(None, 90, 5, 11)
temporal_ctx = tf.keras.backend.sum(combined, axis=-1) # shape=(None, 90, 5)

historical_features = temporal_ctx # shape=(None, 90, 5)
historical_flags = sparse_weights # shape=(None, 90, 1, 11)

###### (2) future data

In [44]:
# (2) future data
future_inputs

<tf.Tensor 'strided_slice_45:0' shape=(None, 30, 5, 8) dtype=float32>

In [45]:
trans_emb_list = []
for i in range(num_inputs):
        grn_output = gated_residual_network(
            historical_inputs[Ellipsis, i],
            hidden_layer_size,
            dropout_rate=dropout_rate,
            use_time_distributed=True)
        trans_emb_list.append(grn_output) # 11个shape=(None, 90, 5)
transformed_embedding = tf.keras.backend.stack(trans_emb_list, axis=-1) # shape=(None, 90, 5, 11)

combined = tf.keras.layers.Multiply()(
          [sparse_weights, transformed_embedding]) # shape=(None, 90, 5, 11)
temporal_ctx = tf.keras.backend.sum(combined, axis=-1) # shape=(None, 90, 5)

historical_features = temporal_ctx # shape=(None, 90, 5)
historical_flags = sparse_weights # shape=(None, 90, 1, 11)

In [46]:
_, future_time_steps, embedding_dim, num_inputs = future_inputs.get_shape().as_list()

flatten = tf.keras.backend.reshape(future_inputs,
                          [-1, future_time_steps, embedding_dim * num_inputs]) # shape=(None, 30, 40)
expanded_static_context = tf.keras.backend.expand_dims(
          static_context_variable_selection, axis=1) # shape=(None, 1, 5)

# Variable selection weights
mlp_outputs, static_gate = gated_residual_network(
          flatten,
          hidden_layer_size,
          output_size=num_inputs,
          dropout_rate=dropout_rate,
          use_time_distributed=True,
          additional_context=expanded_static_context, # here
          return_gate=True) # mlp_outputs: shape=(None, 90, 11); static_gate: shape=(None, 30，8)
sparse_weights = tf.keras.layers.Activation('softmax')(mlp_outputs)
sparse_weights = tf.expand_dims(sparse_weights, axis=2) # hape=(None, 30, 1, 8)

# Non-linear Processing & weight application
trans_emb_list = []
for i in range(num_inputs):
        grn_output = gated_residual_network(
            future_inputs[Ellipsis, i],
            hidden_layer_size,
            dropout_rate=dropout_rate,
            use_time_distributed=True)
        trans_emb_list.append(grn_output) # 8个shape=(None, 30, 5)
transformed_embedding = tf.keras.backend.stack(trans_emb_list, axis=-1) # shape=(None, 30, 5, 8)
combined = tf.keras.layers.Multiply()(
          [sparse_weights, transformed_embedding]) # shape=(None, 30, 5, 8)
temporal_ctx = tf.keras.backend.sum(combined, axis=-1)

future_features = temporal_ctx # shape=(None, 30, 5)
future_flags = sparse_weights # shape=(None, 30, 1, 8)

##### LSTM encoder & decoder

In [47]:
# (1) historical data
lstm = tf.keras.layers.LSTM(
            hidden_layer_size,
            return_sequences=True,
            return_state=True, # diff
            stateful=False,
            # Additional params to ensure LSTM matches CuDNN, See TF 2.0 :
            # (https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM)
            activation='tanh',
            recurrent_activation='sigmoid',
            recurrent_dropout=0,
            unroll=False,
            use_bias=True)

history_lstm, state_h, state_c \
    = lstm(historical_features,initial_state=[static_context_state_h, # h_0
                                               static_context_state_c]) # c_0
                                               
# history_lstm: shape=(None, 90, 5)
# state_h: shape=(None, 5)
# state_c: shape=(None, 5)

In [48]:
state_c[1]

<tf.Tensor 'strided_slice_85:0' shape=(5,) dtype=float32>

In [49]:
# (2) future data
lstm = tf.keras.layers.LSTM(
            hidden_layer_size,
            return_sequences=True,
            return_state=False, # diff
            stateful=False,
            # Additional params to ensure LSTM matches CuDNN, See TF 2.0 :
            # (https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM)
            activation='tanh',
            recurrent_activation='sigmoid',
            recurrent_dropout=0,
            unroll=False,
            use_bias=True)

future_lstm = lstm(
        future_features, 
        initial_state=[state_h, state_c]
        )
# future_lstm: shape=(None, 30, 5)

In [50]:
# (3) combine
lstm_layer = tf.keras.backend.concatenate([history_lstm, future_lstm], axis=1) # shape=(None, 120, 5)

##### Apply gated skip connection

In [51]:
input_embeddings = tf.keras.backend.concatenate([historical_features, future_features], axis=1) # shape=(None, 120, 5)

lstm_layer, _ = apply_gating_layer(
        lstm_layer, hidden_layer_size, dropout_rate, activation=None) # shape=(None, 120, 5)
temporal_feature_layer = add_and_norm([lstm_layer, input_embeddings]) # shape=(None, 120, 5)

##### Static enrichment layers 

In [110]:
## add static_context_enrichment
expanded_static_context = tf.expand_dims(static_context_enrichment, axis=1) # (None, 1, 5) 
enriched, _ = gated_residual_network(
        temporal_feature_layer,
        hidden_layer_size,
        dropout_rate=dropout_rate,
        use_time_distributed=True,
        additional_context=expanded_static_context,
        return_gate=True) # (None, 120, 5)

In [111]:
enriched

<tf.Tensor 'layer_normalization_62/Identity:0' shape=(None, 120, 5) dtype=float32>

##### Temporal Self-Attention Layer

n_head, d_model, dropout
(num_heads, hidden_layer_size, dropout=dropout_rate)
q, k, v, mask=None
(enriched, enriched, enriched,mask=mask)

In [53]:
# causal mask to apply for self-attention layer
len_s = tf.shape(enriched)[1]  # 120
bs = tf.shape(enriched)[:1] # (None,)
mask = K.cumsum(tf.eye(len_s, batch_shape=bs), 1) # shape=(120, 120)

d_k = d_v = hidden_layer_size // num_heads  # 5//4=1
qs_layers = []
ks_layers = []
vs_layers = []

# Use same value layer to facilitate interp
vs_layer = Dense(d_v, use_bias=False)

for _ in range(num_heads):
  qs_layers.append(Dense(d_k, use_bias=False)) # since # of queries = # of keys
  ks_layers.append(Dense(d_k, use_bias=False)) # output_dim = d_k
  vs_layers.append(Dense(d_v, use_bias=False))  # output_dim = d_v

#attention = ScaledDotProductAttention()
heads = []
attns = []
for i in range(num_heads):
  qs = qs_layers[i](enriched) # Q, (None, 120, 1)
  ks = ks_layers[i](enriched)
  vs = vs_layers[i](enriched)

  #head, attn = attention(qs, ks, vs, mask)
  temper = tf.sqrt(tf.cast(tf.shape(ks)[-1], dtype='float32')) # cast: type conversion # shape=()
  # tf.keras.ops.shape(ks)[-1] refers to d_attn(in the paper) = 1
  
  ###################### (QK^T)/sqrt(d_attn) #########################
  attn = Lambda(lambda x: K.batch_dot(x[0], x[1], axes=[2, 2]) / temper)(
        [qs, ks])
  # equals to:
  # K.batch_dot(qs, ks, axes=[2, 2]) / temper
  # output_shape=(None, 120, 120)
  # # shape=(batch, q, k)


  if mask is not None:
      mmask = Lambda(lambda x: (-1e+9) * (1. - tf.cast(x, 'float32')))(
          mask)  # setting to infinity, # 将 mask=1 的位置设为一个很大的负数，softmax 后趋近于0
      attn = Add()([attn, mmask])

  attn = Dropout(0.0)(Activation('softmax')(attn)) # shape=(None, 120, 120)
  # Dropout(0.0): 不丢弃任何神经元，相当于没有起作用，网络将正常传递所有的神经元输出。

  ###################### Attention(Q, K, V) = A(Q, K)V #########################
  head_temp = Lambda(lambda x: K.batch_dot(x[0], x[1]))([attn, vs]) # shape=(None, 120, 1)
  # Wrong edition:
  # head_temp = tf.expand_dims(K.batch_dot(attn, v[0,:]),axis=0) # shape=(1, 120, 5)
  # equals to:
  # output = Lambda(lambda x: K.batch_dot(x[0], x[1]))([attn, enriched])

  head_dropout = Dropout(dropout_rate)(head_temp) # shape=(None, 120, 1)
  
  
  heads.append(head_dropout) # 4个shape=(1, 120, 5)
  attns.append(attn) # 4个shape=(1, 120, 120)
  
###################### Multi-Head #########################
head = tf.keras.backend.stack(heads) if num_heads > 1 else heads[0] # (4, None, 120, 1)

# Result 2: self_att      shape=(4, None, 120, 120)
self_att = tf.keras.backend.stack(attns)

# Result 1: x   shape=(None, 120, 5)
outputs = K.mean(head, axis=0) if num_heads > 1 else head # (None, 120, 1)
w_o = Dense(hidden_layer_size, use_bias=False)
outputs = w_o(outputs) # input_dim=d_model; output_dim=K.mean(head, axis=0)
outputs = Dropout(dropout_rate)(outputs)  # output dropout
x = outputs

x, _ = apply_gating_layer(
        x,
        hidden_layer_size,
        dropout_rate=dropout_rate,
        activation=None)

In [54]:
x = add_and_norm([x, enriched])
print(x)
print(enriched)

Tensor("layer_normalization_48/Identity:0", shape=(None, 120, 5), dtype=float32)
Tensor("layer_normalization_47/Identity:0", shape=(None, 120, 5), dtype=float32)


Decoder self attention
x, self_att \
        = InterpretableMultiHeadAttention(
        num_heads, hidden_layer_size, dropout=dropout_rate)(enriched, enriched, enriched,
                          mask=mask)
class InterpretableMultiHeadAttention():

    self.d_k = self.d_v = d_k = d_v = hidden_layer_size // num_heads
    self.qs_layers = []
    self.ks_layers = []
    self.vs_layers = []

    # Use same value layer to facilitate interp
    vs_layer = Dense(d_v, use_bias=False)

    for _ in range(n_head):
      self.qs_layers.append(Dense(d_k, use_bias=False)) # since # of queries = # of keys
      self.ks_layers.append(Dense(d_k, use_bias=False)) # output_dim = d_k
      self.vs_layers.append(Dense(d_v, use_bias=False))  # output_dim = d_v


    heads = []
    attns = []
    for i in range(n_head):
      qs = self.qs_layers[i](enriched) # input_dim=q; output_dim = d_k
      ks = self.ks_layers[i](enriched)
      vs = self.vs_layers[i](enriched)
      head, attn = ScaledDotProductAttention()(qs, ks, vs, mask)
      head_dropout = Dropout(self.dropout)(head)
      
      heads.append(head_dropout)
      attns.append(attn)


    temper = tf.sqrt(tf.cast(tf.shape(ks)[-1], dtype='float32'))
    attn = Lambda(lambda x: K.batch_dot(x[0], x[1], axes=[2, 2]) / temper)(
        [qs, ks])  # shape=(batch, q, k)
    if mask is not None:
      mmask = Lambda(lambda x: (-1e+9) * (1. - K.cast(x, 'float32')))(
          mask)  # setting to infinity
      attn = Add()([attn, mmask])
    attn = Activation('softmax')(attn)
    attn = Dropout(attn_dropout=0.0)(attn)
    output = Lambda(lambda x: K.batch_dot(x[0], x[1]))([attn, vs])

head = K.stack(heads) if n_head > 1 else heads[0]
    attn = K.stack(attns)

    outputs = K.mean(head, axis=0) if n_head > 1 else head
    outputs = Dense(d_model, use_bias=False)(outputs) # input_dim=d_model; output_dim=K.mean(head, axis=0)
    outputs = Dropout(self.dropout)(outputs)  # output dropout


##### Position-wise Feed-forward Layer

In [55]:
decoder = gated_residual_network(
        x,
        hidden_layer_size,
        dropout_rate=dropout_rate,
        use_time_distributed=True)
decoder

<tf.Tensor 'layer_normalization_49/Identity:0' shape=(None, 120, 5) dtype=float32>

##### Final

In [56]:
# Final skip connection
decoder, _ = apply_gating_layer(
        decoder, hidden_layer_size, activation=None)
transformer_layer = add_and_norm([decoder, temporal_feature_layer])

# Attention components for explainability
attention_components = {
        # Temporal attention weights
        'decoder_self_attn': self_att,
        # Static variable selection weights
        'static_flags': static_weights[Ellipsis, 0],
        # Variable selection weights of past inputs
        'historical_flags': historical_flags[Ellipsis, 0, :],
        # Variable selection weights of future inputs
        'future_flags': future_flags[Ellipsis, 0, :]
    }

# what we want:
# transformer_layer: shape=(None, 120, 5)
# all_inputs
# attention_components

In [57]:
transformer_layer

<tf.Tensor 'layer_normalization_50/Identity:0' shape=(None, 120, 5) dtype=float32>

In [58]:
_attention_components =  attention_components
attention_components

{'decoder_self_attn': <tf.Tensor 'stack_8:0' shape=(4, None, 120, 120) dtype=float32>,
 'static_flags': <tf.Tensor 'strided_slice_92:0' shape=(None, 9) dtype=float32>,
 'historical_flags': <tf.Tensor 'strided_slice_93:0' shape=(None, 90, 11) dtype=float32>,
 'future_flags': <tf.Tensor 'strided_slice_94:0' shape=(None, 30, 8) dtype=float32>}

In [59]:
# Dense
outputs = tf.keras.layers.TimeDistributed(
          tf.keras.layers.Dense(output_size * len(quantiles))) \
          (transformer_layer[Ellipsis, num_encoder_steps:, :])
# shape=(None, 30, 3)

In [139]:
tf.keras.layers.TimeDistributed(
          tf.keras.layers.Dense(output_size * len(quantiles))) \
          (transformer_layer[Ellipsis, num_encoder_steps:, :])
# shape=(None, 30, 3)

<tf.Tensor 'time_distributed_164/Identity:0' shape=(None, 30, 3) dtype=float32>

##### Model

In [60]:
#_attention_components = attention_components
model = tf.keras.Model(inputs=all_inputs, outputs=outputs) # all_inputs: ([None, 120, 20])
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 120, 20)]    0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None, 120, 14)]    0           input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_2 (Te [(None, 120)]        0           tf_op_layer_strided_slice_1[0][0]
__________________________________________________________________________________________________
tf_op_layer_strided_slice_3 (Te [(None, 120)]        0           tf_op_layer_strided_slice_1[0][0]
______________________________________________________________________________________________

In [62]:
valid_quantiles = quantiles

In [63]:
class QuantileLossCalculator(object):
        """Computes the combined quantile loss for prespecified quantiles.

        Attributes:
          quantiles: Quantiles to compute losses
        """

        def __init__(self, quantiles):
          """Initializes computer with quantiles for loss calculations.

          Args:
            quantiles: Quantiles to use for computations.
          """
          

        def quantile_loss(self, a, b):
          """Returns quantile loss for specified quantiles.

          Args:
            a: Targets
            b: Predictions
          """
          quantiles_used = set(quantiles)

          loss = 0.
          for i, quantile in enumerate(valid_quantiles):
            if quantile in quantiles_used:
              loss += utils.tensorflow_quantile_loss(
                  a[Ellipsis, output_size * i:output_size * (i + 1)],
                  b[Ellipsis, output_size * i:output_size * (i + 1)], quantile)
          return loss


In [64]:
quantile_loss = QuantileLossCalculator(valid_quantiles).quantile_loss
adam = tf.keras.optimizers.Adam(
          learning_rate=learning_rate, clipnorm=max_gradient_norm)
model.compile(
          loss=quantile_loss, optimizer=adam, sample_weight_mode='temporal')
# model.compile()方法用于在配置训练方法时，告知训练时用的优化器、损失函数和准确率评测标准
# sample_weight_mode='temporal' is very important, we need to assign different weight for each timestamp

_input_placeholder = all_inputs

def build_model(self):
    """Build model and defines training losses.

    Returns:
      Fully defined Keras model.
    """

    #with tf.variable_scope(name):
    with tf.compat.v1.variable_scope(name):

      transformer_layer, all_inputs, attention_components \
          = _build_base_graph()
      
      # Dense
      outputs = tf.keras.layers.TimeDistributed(
          tf.keras.layers.Dense(output_size * len(quantiles))) \
          (transformer_layer[Ellipsis, num_encoder_steps:, :])

      _attention_components = attention_components

      adam = tf.keras.optimizers.Adam(
          learning_rate=learning_rate, clipnorm=max_gradient_norm)

      model = tf.keras.Model(inputs=all_inputs, outputs=outputs)

      print(model.summary())

      valid_quantiles = quantiles
      output_size = output_size
      
      quantile_loss = QuantileLossCalculator(valid_quantiles).quantile_loss

      model.compile(
          loss=quantile_loss, optimizer=adam, sample_weight_mode='temporal')

      _input_placeholder = all_inputs

    return model

#### Sample training & validating data
##### Read data

In [66]:
train = pd.read_csv('train_tf.csv')
valid = pd.read_csv('valid_tf.csv')
test = pd.read_csv('test_tf.csv')

In [67]:
test.head()

,store_nbr,item_nbr,unit_sales,onpromotion,traj_id,unique_id,open,date,log_sales,oil,...,family,class,perishable,transactions,day_of_week,day_of_month,month,national_hol,regional_hol,local_hol
0,0,0,3.0,2,1_103520,1_103520_2013-09-05 00:00:00,1.0,2013-09-05,0.186498,0.989231,...,1,0,0,0.519535,3,5,9,34,1,3
1,0,0,4.0,2,1_103520,1_103520_2013-09-06 00:00:00,1.0,2013-09-06,0.587660,1.037158,...,1,0,0,0.715138,4,6,9,34,1,3
2,0,0,2.0,2,1_103520,1_103520_2013-09-07 00:00:00,1.0,2013-09-07,-0.378907,-1.486235,...,1,0,0,-0.253098,5,7,9,34,1,3
3,0,0,1.0,2,1_103520,1_103520_2013-09-09 00:00:00,1.0,2013-09-09,-1.345475,1.014551,...,1,0,0,0.620596,0,9,9,34,1,3
4,0,0,2.0,2,1_103520,1_103520_2013-09-10 00:00:00,1.0,2013-09-10,-0.378907,0.966172,...,1,0,0,0.452703,1,10,9,34,1,3


In [143]:
print(train.shape)
print(valid.shape)
print(test.shape)

(921, 24)
(360, 24)
(360, 24)


##### Function

In [68]:
def _get_single_col_by_type(input_type):
    """Returns name of single column for input type."""
    return utils.get_single_col_by_input_type(input_type,
                                              column_definition)

In [204]:
identifier = '1_103520'
df = train[train['traj_id']=='1_103520']
num_entries = len(df) # num_entries=246
valid_sampling_locations = []
split_train_map = {}
if num_entries >= time_steps: # time_steps=120
  valid_sampling_locations += [(identifier, time_steps + i) for i in range(num_entries - time_steps + 1)]
  split_train_map[identifier] = df
# valid_sampling_locations: for each each traj_id, every day after 120th Day
# split_train_map = {traj_id : df}  

inputs = np.zeros((300, 120, input_size), dtype=object) # dtype=object: trainframe contains diff formats
outputs = np.zeros((300, 120, output_size), dtype=object)
time = np.empty((300, 120, 1), dtype=object)
identifiers = np.empty((300, 120, 1), dtype=object)

# len(ranges) <= 300, ranges=[('1_103520', 120),('1_103520', 121),...('1_103520', 246)]]
if 300 > 0 and len(valid_sampling_locations) > 300:
  print('Extracting {} samples...'.format(300))
  ranges = [
  valid_sampling_locations[i] for i in np.random.choice( 
  len(valid_sampling_locations), 300, replace=False)
  ] # Random pick 300 from len(valid_sampling_locations)
else:
  print('Max samples={} exceeds # available segments={}'.format(
  300, len(valid_sampling_locations)))
  ranges = valid_sampling_locations

id_col = _get_single_col_by_type(InputTypes.ID) # 'traj_id'
time_col = _get_single_col_by_type(InputTypes.TIME) # 'date'
target_col = _get_single_col_by_type(InputTypes.TARGET) # 'log_sales'
input_cols = [ # other columns in column_definition
  tup[0]
  for tup in column_definition
  if tup[2] not in {InputTypes.ID, InputTypes.TIME}
]

for i, tup in enumerate(ranges): # eg: i=0, tup = ('1_103520', 120)
  if (i + 1 % 1000) == 0:
    print(i + 1, 'of', max_samples, 'samples done...')
  identifier, start_idx = tup
  sliced = split_train_map[identifier].iloc[start_idx - 120:start_idx]

Max samples=300 exceeds # available segments=127


In [205]:
i=126
tup = ('1_103520', 246)
identifier, start_idx = tup # identifier='1_103520',start_idx=120
sliced = split_train_map[identifier].iloc[start_idx - 120:start_idx]
inputs[i, :, :] = sliced[input_cols] # 填在(1, 120, 20)
outputs[i, :, :] = sliced[[target_col]] # (1, 120, 1)
time[i, :, 0] = sliced[time_col]
identifiers[i, :, 0] = sliced[id_col] # since time and identifier are 1-D variable

sampled_data = {
'inputs': inputs, # (300, 120, 20)
'outputs': outputs[:, 90:, :], # (300,30,1)
'active_entries': np.ones_like(outputs[:, 90:, :]), # (300,30,1)
'time': time, # (300, 120, 1)
'identifier': identifiers
}

In [69]:
def _batch_sampled_data(data, max_samples):
    """Samples segments into a compatible format.

    Args:
      data: Sources data to sample and batch
      max_samples: Maximum number of samples in batch

    Returns:
      Dictionary of batched data with the maximum samples specified.
    """

    if max_samples < 1:
      raise ValueError(
          'Illegal number of samples specified! samples={}'.format(max_samples))

    id_col = _get_single_col_by_type(InputTypes.ID) # 'traj_id'
    time_col = _get_single_col_by_type(InputTypes.TIME) # 'date'

    data.sort_values(by=[id_col, time_col], inplace=True)

    print('Getting valid sampling locations.')
    valid_sampling_locations = []
    split_data_map = {}
    for identifier, df in data.groupby(id_col): # for each traj_id
      print('Getting locations for {}'.format(identifier))
      num_entries = len(df)
      if num_entries >= time_steps:
        valid_sampling_locations += [
            (identifier, time_steps + i)
            for i in range(num_entries - time_steps + 1)
        ]
      split_data_map[identifier] = df
    # valid_sampling_locations: for each each traj_id, every day after 120th Day
    # split_data_map = {traj_id : df}  

    inputs = np.zeros((max_samples, time_steps, input_size), dtype=object) # dtype=object: dataframe contains diff formats
    outputs = np.zeros((max_samples, time_steps, output_size), dtype=object)
    time = np.empty((max_samples, time_steps, 1), dtype=object)
    identifiers = np.empty((max_samples, time_steps, 1), dtype=object)

    if max_samples > 0 and len(valid_sampling_locations) > max_samples:
      print('Extracting {} samples...'.format(max_samples))
      ranges = [
          valid_sampling_locations[i] for i in np.random.choice( 
              len(valid_sampling_locations), max_samples, replace=False)
      ] # Random pick max_samples from len(valid_sampling_locations)
    else:
      print('Max samples={} exceeds # available segments={}'.format(
          max_samples, len(valid_sampling_locations)))
      ranges = valid_sampling_locations

    id_col = _get_single_col_by_type(InputTypes.ID) # 'traj_id'
    time_col = _get_single_col_by_type(InputTypes.TIME) # 'date'
    target_col = _get_single_col_by_type(InputTypes.TARGET) # 'log_sales'
    input_cols = [ # other columns in column_definition
        tup[0]
        for tup in column_definition
        if tup[2] not in {InputTypes.ID, InputTypes.TIME}
    ]

    for i, tup in enumerate(ranges):
      if (i + 1 % 1000) == 0:
        print(i + 1, 'of', max_samples, 'samples done...')
      identifier, start_idx = tup
      sliced = split_data_map[identifier].iloc[start_idx -
                                               time_steps:start_idx]
      inputs[i, :, :] = sliced[input_cols]
      outputs[i, :, :] = sliced[[target_col]]
      time[i, :, 0] = sliced[time_col]
      identifiers[i, :, 0] = sliced[id_col]

    sampled_data = {
        'inputs': inputs,
        'outputs': outputs[:, num_encoder_steps:, :],
        'active_entries': np.ones_like(outputs[:, num_encoder_steps:, :]),
        'time': time,
        'identifier': identifiers
    }

    return sampled_data

In [144]:
train_samples = 300
valid_samples = 100
_batch_sampled_data(train, max_samples=train_samples)['inputs']

Getting valid sampling locations.
Getting locations for 1_103520
Getting locations for 1_103665
Getting locations for 1_96995
Getting locations for 25_103665
Extracting 300 samples...


array([[[-1.3454751863495475, 0.6189661245892091, 0.6675337147068836,
         ..., 31.0, 1.0, 3.0],
        [-0.3789074381700053, 0.5619152072903433, 0.6978270901535721,
         ..., 34.0, 1.0, 3.0],
        [-1.3454751863495475, 0.6026658625038188, 0.6978270901535721,
         ..., 34.0, 1.0, 3.0],
        ...,
        [0.1864984489215168, 0.6841671729307701, 0.7269901157701603,
         ..., 34.0, 1.0, 3.0],
        [-1.3454751863495475, 0.5668052859159604, 0.7213383666196587,
         ..., 34.0, 1.0, 3.0],
        [-1.3454751863495475, 0.5048642899914775, 0.758413841046949,
         ..., 34.0, 1.0, 3.0]],

       [[-1.3454751863495475, -1.1593924689268649, -1.4862348515662502,
         ..., 34.0, 1.0, 3.0],
        [0.1864984489215168, -1.2783843821502134, 0.7021224195079532,
         ..., 34.0, 1.0, 3.0],
        [-0.3789074381700053, -1.2816444345672915, 0.69036678127491,
         ..., 34.0, 1.0, 3.0],
        ...,
        [1.1530661971010592, -0.6410441346114555, 0.883656602222

In [70]:
def _batch_data(data):
    """Batches data for training.

    Converts raw dataframe from a 2-D tabular format to a batched 3-D array
    to feed into Keras model.

    Args:
      data: DataFrame to batch

    Returns:
      Batched Numpy array with shape=(?, time_steps, input_size)
    """

    # Functions.
    def _batch_single_entity(input_data):
      time_steps = len(input_data)
      lags = time_steps
      x = input_data.values
      if time_steps >= lags:
        return np.stack(
            [x[i:time_steps - (lags - 1) + i, :] for i in range(lags)], axis=1)

      else:
        return None

    id_col = _get_single_col_by_type(InputTypes.ID)
    time_col = _get_single_col_by_type(InputTypes.TIME)
    target_col = _get_single_col_by_type(InputTypes.TARGET)
    input_cols = [
        tup[0]
        for tup in column_definition
        if tup[2] not in {InputTypes.ID, InputTypes.TIME}
    ]

    data_map = {}
    for _, sliced in data.groupby(id_col):

      col_mappings = {
          'identifier': [id_col],
          'time': [time_col],
          'outputs': [target_col],
          'inputs': input_cols
      }

      for k in col_mappings:
        cols = col_mappings[k]
        arr = _batch_single_entity(sliced[cols].copy())

        if k not in data_map:
          data_map[k] = [arr]
        else:
          data_map[k].append(arr)

    # Combine all data
    for k in data_map:
      data_map[k] = np.concatenate(data_map[k], axis=0)

    # Shorten target so we only get decoder steps
    data_map['outputs'] = data_map['outputs'][:, num_encoder_steps:, :]

    active_entries = np.ones_like(data_map['outputs'])
    if 'active_entries' not in data_map:
      data_map['active_entries'] = active_entries
    else:
      data_map['active_entries'].append(active_entries)

    return data_map

In [71]:
class TFTDataCache(object):
  """Caches data for the TFT.""" 
  # stores multiple copies of data or files in a temporary storage location—or cache—
  # so they can be accessed faster

  _data_cache = {}

  @classmethod
  def update(cls, data, key):  # cls is similar to self
    """Updates cached data.

    Args:
      data: Source to update
      key: Key to dictionary location
    """
    cls._data_cache[key] = data

  @classmethod
  def get(cls, key):
    """Returns data stored at key location."""
    return cls._data_cache[key].copy()

  @classmethod
  def contains(cls, key): # TFTDataCache.contains('train') and TFTDataCache.contains('valid')
    """Retuns boolean indicating whether key is present in cache."""
    return key in cls._data_cache

##### Sample

In [74]:
# model.cache_batched_data(train, "train", num_samples=train_samples)
if train_samples > 0:
      TFTDataCache.update(
          _batch_sampled_data(train, max_samples=train_samples),
            "train")  
      # create a temperal set, where key='train', value=_batch_sampled_data(train, max_samples=train_samples)
      # _batch_sampled_data(train, max_samples=train_samples)
      # dict_keys(['inputs', 'outputs', 'active_entries', 'time', 'identifier'])
else:
      TFTDataCache.update(
            _batch_data(train), 
            "train")
      
# model.cache_batched_data(valid, "valid", num_samples=valid_samples)
if valid_samples > 0:
      TFTDataCache.update(
          _batch_sampled_data(valid, max_samples=valid_samples),
            "valid")  
      # create a temperal set, where key='valid', value=_batch_sampled_data(valid, max_samples=valid_samples)
else:
      TFTDataCache.update(
            _batch_data(valid), 
            "valid")

Getting valid sampling locations.
Getting locations for 1_103520
Getting locations for 1_103665
Getting locations for 1_96995
Getting locations for 25_103665
Extracting 300 samples...
Getting valid sampling locations.
Getting locations for 1_103520
Getting locations for 1_103665
Getting locations for 25_103665
Max samples=100 exceeds # available segments=3


In [75]:
print('Getting batched_data')
train_df=None
valid_df=None
if train_df is None:
      print('Using cached training data')
      train_data = TFTDataCache.get('train')
else:
      train_data = _batch_data(train_df)

if valid_df is None:
      print('Using cached validation data')
      valid_data = TFTDataCache.get('valid')
else:
      valid_data = _batch_data(valid_df)

Getting batched_data
Using cached training data
Using cached validation data


In [76]:
print('Using keras standard fit')
# Unpack without sample weights
data = train_data['inputs'] # (300, 120, 20)
labels = train_data['outputs'] # (300, 30, 1)
active_flags = (np.sum(train_data['active_entries'], axis=-1) > 0.0) * 1.0  # (300, 30)
 
val_data = valid_data['inputs']
val_labels = valid_data['outputs']
val_flags = (np.sum(valid_data['active_entries'], axis=-1) > 0.0) * 1.0

Using keras standard fit


In [77]:
# Add relevant callbacks
name = 'favorita'
def get_keras_saved_path(model_folder):
    """Returns path to keras checkpoint."""
    return os.path.join(model_folder, '{}.check'.format(name))
callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=early_stopping_patience,
            min_delta=1e-4),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=get_keras_saved_path(_temp_folder),
            monitor='val_loss',
            save_best_only=True,
            save_weights_only=True),
        tf.keras.callbacks.TerminateOnNaN()
    ]

In [78]:
print('Getting batched_data')
if train_df is None:
  print('Using cached training data')
  train_data = TFTDataCache.get('train')
else:
  train_data = _batch_data(train_df)

if valid_df is None:
  print('Using cached validation data')
  valid_data = TFTDataCache.get('valid')
else:
  valid_data = _batch_data(valid_df)

print('Using keras standard fit')


Getting batched_data
Using cached training data
Using cached validation data
Using keras standard fit


In [79]:
def _get_active_locations(x):
    """Formats sample weights for Keras training."""
    return (np.sum(x, axis=-1) > 0.0) * 1.0

def _unpack(data):
  return data['inputs'], data['outputs'], \
  _get_active_locations(data['active_entries'])

# Unpack without sample weights
data, labels, active_flags = _unpack(train_data)
val_data, val_labels, val_flags = _unpack(valid_data)

all_callbacks = callbacks

##### Fit

In [80]:
labels.shape

(300, 30, 1)

In [81]:
model.fit(
        x=tf.convert_to_tensor(data,dtype=np.float32), # shape=(300, 120, 20)
        y=np.concatenate([labels, labels, labels], axis=-1), # shape=(300, 30, 3) # 3 represents # of quantiles
        #y=tf.convert_to_tensor(np.concatenate([labels, labels, labels], axis=-1),dtype=np.float32), 
        sample_weight=active_flags,
        epochs=num_epochs,
        batch_size=minibatch_size,
        validation_data=(val_data,
                         np.concatenate([val_labels, val_labels, val_labels],
                                        axis=-1), val_flags),
        callbacks=all_callbacks,
        shuffle=True,
        use_multiprocessing=True,
        workers=n_multiprocessing_workers
        )

# use_multiprocessing: https://stackoverflow.com/questions/52932406/is-the-class-generator-inheriting-sequence-thread-safe-in-keras-tensorflow/63641535#63641535

Train on 300 samples, validate on 100 samples
300/300 [==============================] - 10s 32ms/sample - loss: 2.3337 - val_loss: 0.4742


In [82]:
# Load best checkpoint again
tmp_checkpont = get_keras_saved_path(_temp_folder)
if os.path.exists(tmp_checkpont):
  load(
  _temp_folder,
  use_keras_loadings=True)

else:
  print('Cannot load from {}, skipping ...'.format(_temp_folder))

Cannot load from 0615_result\tmp, skipping ...


#### Evaluate
"""Applies evaluation metric to the training data.

    Args:
      data: Dataframe for evaluation
      eval_metric: Evaluation metic to return, based on model definition.

    Returns:
      Computed evaluation loss.
    """

In [83]:
print('Using cached validation data')
raw_data = TFTDataCache.get('valid')
# _batch_data(data)

inputs = raw_data['inputs']
outputs = raw_data['outputs']
active_entries = _get_active_locations(raw_data['active_entries'])

metric_values = model.evaluate(
    x=inputs,
    y=np.concatenate([outputs, outputs, outputs], axis=-1),
    sample_weight=active_entries,
    workers=16,
    use_multiprocessing=True)

metrics = pd.Series(metric_values, model.metrics_names)
print(metrics['loss'])

Using cached validation data
100/100 [==============================] - 0s 1ms/sample - loss: 0.4742
0.47415144205093385


#### Predict
"""Computes predictions for a given input dataset.

    Args:
      df: Input dataframe
      return_targets: Whether to also return outputs aligned with predictions to
        faciliate evaluation

    Returns:
      Input dataframe or tuple of (input dataframe, algined output dataframe).
    """

In [84]:
data = _batch_data(test)
inputs = data['inputs']
time = data['time']
identifier = data['identifier']
outputs = data['outputs']

combined = model.predict(inputs,
                         workers=16,
                         use_multiprocessing=True,
                         batch_size=minibatch_size)
# Format output_csv
if output_size != 1:
    raise NotImplementedError('Current version only supports 1D targets!')

def format_outputs(prediction):
      """Returns formatted dataframes for prediction."""

      flat_prediction = pd.DataFrame(
          prediction[:, :, 0],
          columns=[
              't+{}'.format(i)
              for i in range(time_steps - num_encoder_steps)
          ])
      cols = list(flat_prediction.columns)
      flat_prediction['forecast_time'] = time[:, num_encoder_steps - 1, 0]
      flat_prediction['identifier'] = identifier[:, 0, 0]

      # Arrange in order
      return flat_prediction[['forecast_time', 'identifier'] + cols]
# Extract predictions for each quantile into different entries
process_map = {
        'p{}'.format(int(q * 100)):
        combined[Ellipsis, i * output_size:(i + 1) * output_size]
        for i, q in enumerate(quantiles)
    }
return_targets = False
if return_targets:
      # Add targets if relevant
      process_map['targets'] = outputs

In [85]:
print({k: format_outputs(process_map[k]) for k in process_map})

{'p10':   forecast_time identifier       t+0       t+1       t+2       t+3       t+4  \
0    2014-02-01   1_103520 -0.921138 -0.975129 -0.858840 -0.950917 -1.011550   
1    2014-01-13   1_103665 -0.663760 -0.763067 -0.881042 -0.723420 -0.942091   
2    2014-01-10  25_103665 -0.411772 -0.710101 -0.678902 -0.659965 -0.801824   

        t+5       t+6       t+7  ...      t+20      t+21      t+22      t+23  \
0 -0.827111 -0.973458 -0.991614  ... -0.891888 -0.567603 -0.584162 -0.750056   
1 -0.867548 -0.896401 -0.795341  ... -0.620795 -0.728647 -0.564356 -0.657984   
2 -0.624304 -0.790431 -0.726994  ... -0.596657 -0.762090 -0.524362 -0.793863   

       t+24      t+25      t+26      t+27      t+28      t+29  
0 -0.538907 -0.803590 -0.717859 -0.751176 -0.647569 -0.644404  
1 -0.670879 -0.766426 -0.585354 -0.720180 -0.658800 -0.671636  
2 -0.701905 -0.634675 -0.616152 -0.759249 -0.531140 -0.788158  

[3 rows x 32 columns], 'p50':   forecast_time identifier       t+0       t+1       t+2       

#### Get Attention
 """Computes TFT attention weights for a given dataset.

    Args:
      df: Input dataframe

    Returns:
        Dictionary of numpy arrays for temporal attention weights and variable
          selection weights, along with their identifiers and time indices
    """

In [86]:
for k in _attention_components:
    print(_attention_components[k],{_input_placeholder : batched_inputs[0].astype(np.float32)})

NameError: name 'batched_inputs' is not defined

In [472]:
_input_placeholder

<tf.Tensor 'input_46:0' shape=(None, 120, 20) dtype=float32>

In [468]:
batched_inputs[0].astype(np.float32)

array([[[ 0.18649845,  0.5195345 ,  0.9892313 , ..., 34.        ,
          1.        ,  3.        ],
        [ 0.5876603 ,  0.71513766,  1.0371581 , ..., 34.        ,
          1.        ,  3.        ],
        [-0.37890744, -0.2530979 , -1.4862349 , ..., 34.        ,
          1.        ,  3.        ],
        ...,
        [-1.3454752 ,  0.71839774,  0.80362785, ..., 34.        ,
          1.        ,  3.        ],
        [-1.3454752 ,  0.7917489 ,  0.75841385, ..., 34.        ,
          1.        ,  3.        ],
        [ 1.3680235 ,  0.5733254 ,  0.7647438 , ..., 34.        ,
          1.        ,  3.        ]],

       [[ 0.18649845,  0.7852288 ,  0.9831274 , ..., 34.        ,
          1.        ,  3.        ],
        [-1.3454752 ,  0.46411362,  0.9385916 , ..., 34.        ,
          1.        ,  3.        ],
        [ 1.7184721 , -0.15692635, -1.4862349 , ..., 34.        ,
          1.        ,  3.        ],
        ...,
        [-0.37890744,  0.66297686,  0.80407995, ..., 3

In [474]:

data = _batch_data(test)
inputs = data['inputs']
identifiers = data['identifier']
time = data['time']

def get_batch_attention_weights(input_batch):
      """Returns weights for a given minibatch of data."""
      input_placeholder = _input_placeholder
      attention_weights = {}
      for k in _attention_components:
        attention_weight = tf.compat.v1.keras.backend.get_session().run(
            _attention_components[k],
            {input_placeholder: input_batch.astype(np.float32)})
        
        attention_weights[k] = attention_weight
      return attention_weights

    # Compute number of batches
batch_size = minibatch_size
n = inputs.shape[0]
num_batches = n // batch_size
if n - (num_batches * batch_size) > 0:
      num_batches += 1

# Split up inputs into batches
batched_inputs = [
        inputs[i * batch_size:(i + 1) * batch_size, Ellipsis]
        for i in range(num_batches)
    ]

# Get attention weights, while avoiding large memory increases
attention_by_batch = [
        get_batch_attention_weights(batch) for batch in batched_inputs
    ]

TypeError: Tensor is unhashable if Tensor equality is enabled. Instead, use tensor.experimental_ref() as the key.

In [455]:
data = _batch_data(test)
inputs = data['inputs']
identifiers = data['identifier']
time = data['time']

def get_batch_attention_weights(input_batch):
      """Returns weights for a given minibatch of data."""
      input_placeholder = _input_placeholder
      attention_weights = {}
      for k in _attention_components:
        attention_weight = tf.keras.backend.get_session().run(
            _attention_components[k],
            {input_placeholder: input_batch.astype(np.float32)})
        attention_weights[k] = attention_weight
      return attention_weights

    # Compute number of batches
batch_size = minibatch_size
n = inputs.shape[0]
num_batches = n // batch_size
if n - (num_batches * batch_size) > 0:
      num_batches += 1

# Split up inputs into batches
batched_inputs = [
        inputs[i * batch_size:(i + 1) * batch_size, Ellipsis]
        for i in range(num_batches)
    ]

# Get attention weights, while avoiding large memory increases
attention_by_batch = [
        get_batch_attention_weights(batch) for batch in batched_inputs
    ]
attention_weights = {}
for k in _attention_components:
      attention_weights[k] = []
      for batch_weights in attention_by_batch:
        attention_weights[k].append(batch_weights[k])

      if len(attention_weights[k][0].shape) == 4:
        tmp = np.concatenate(attention_weights[k], axis=1)
      else:
        tmp = np.concatenate(attention_weights[k], axis=0)

      del attention_weights[k]
      gc.collect()
      attention_weights[k] = tmp

attention_weights['identifiers'] = identifiers[:, 0, 0]
attention_weights['time'] = time[:, :, 0]

print(attention_weights)


AttributeError: module 'tensorflow_core.keras.backend' has no attribute 'get_session'

In [ ]:

      input_placeholder = _input_placeholder
      attention_weights = {}
      for k in _attention_components:
        attention_weight = tf.keras.backend.get_session().run(
            _attention_components[k],
            {input_placeholder: batched_inputs[0].astype(np.float32)})
        attention_weights[k] = attention_weight
      return attention_weights


In [449]:
for k in _attention_components:
    print(k)

decoder_self_attn
static_flags
historical_flags
future_flags


In [452]:
tf.keras.backend.get_session().run(
            _attention_components[_attention_components[0]],
            {input_placeholder: batched_inputs[0].astype(np.float32)})

AttributeError: module 'tensorflow_core.keras.backend' has no attribute 'get_session'